In [ ]:
using System;
using System.IO;
using System.Collections.Generic;
using System.Collections;
using System.Threading;
using System.Text.Json;

In [ ]:
public class Debug : IDisposable
{
    public static bool IsEnabled {get; private set;} = false;

    public static void Print(string input)
    {
        if(IsEnabled)
        {
            Console.WriteLine(input);
        }
    }

    public Debug(bool enable = true)
    {
        IsEnabled = enable;
    }

    public void Dispose()
    {
        IsEnabled = false;
    }
}

In [ ]:
public interface State
{
    bool IsGoal {get;}
    string Id {get;}
    IEnumerable<(State state, long cost)> Neighbours();
}

In [ ]:
public class Dijkstra
{
    public Dictionary<string, long> Cost = new Dictionary<string, long>();
    public PriorityQueue<State, long> Queue = new PriorityQueue<State, long>();
    public int Min = int.MaxValue;
    public State MinState = null;

    public int Count = 0;
    
    
    public (State s, long c) Run(State start)
    {
        Queue.Enqueue(start, 0);

        State node;
        long cost;
        while(Queue.TryDequeue(out node, out cost))
        {
            if(node.Id.Length < Min)
            {
                Min = node.Id.Length;
                MinState = node;
            }
            
            Count++;

            if(Count % 100000 == 0)
            {
                Debug.Print($"Visited {Count} nodes. {Queue.Count} left, Current: {node.Id}, Min: {Min}");
            }

            // if(Count % 10000000 == 0)
            // {
            //     Debug.Print("Breaking");
            //     break;
            // }            
            
            if(node.IsGoal)
            {
                break;
            }
            
            foreach(var n in node.Neighbours())
            {
                long total = n.cost + cost;

                if(Cost.TryGetValue(n.state.Id, out var c))
                {
                    if(c <= total)
                    {
                        continue;
                    }
                }
                Cost[n.state.Id] = total;
                Queue.Enqueue(n.state, total);
            }
        }
        Debug.Print("Done?");

        if(node?.IsGoal == true)
        {
            Debug.Print("YES!");
            return (node, cost);
        }
        else{
            Debug.Print("NO!");
            return (node, -1);
        }
    }
}

In [ ]:
var input = File.ReadAllLines("input.txt");

In [ ]:
public record Replacement
{
    public string From {get; init;}
    public string To {get; init;}

    public Replacement(string input)
    {
        var segments = input.Split("=>", 2, StringSplitOptions.TrimEntries);
        From = segments[0];
        To = segments[1];
    }
}

In [ ]:
static var d = new Dictionary<string, List<string>>();
static var rev = new Dictionary<string, string>();
static var initial = "";

foreach(var line in input)
{
    if(line.Contains("=>"))
    {
        var segments = line.Split("=>", 2, StringSplitOptions.TrimEntries);
        
        if(!d.ContainsKey(segments[0]))
        {
            d.Add(segments[0], new List<string>());
        }
        var r = string.Concat(segments[1].Reverse());
        var rr = string.Concat(segments[0].Reverse());
        d[rr].Add(r);

        rev[r] = rr;
    }
    else if(!string.IsNullOrWhiteSpace(line))
    {
        initial = string.Concat(line.Reverse());
    }
}


In [ ]:
initial

rAgMPBPnRhTiSaCrAFYFnRiSrAgMPaCaCnRPaCaCrAFBnRiSaCBPaChTiSaCrAFrAgMPnRiTnRiShTiShTrAFhTiSaCBrAFrAFYFnRiSaCaCnRiSaCaCaCaCrAFnRPaCBiTPrAFnRrAgMPnRiSaChTiSBPaChTiSrAFYFnRiSaCaCBrAFnRiSaChTiSrAFYFnRiSaCBPBPhTiSBPaChTiSaCaCaCaCBrAlAiSBPnRiSaCaCrAlAiSaCaChTiSaCaCaCaCYgMrAFnRiSBPnRiSaCaCrArAFnRaChTiSYFnRiSaCaCBPrAlAiSnRPaCrAgMiTPBPhTiSaChTiSrAFaCYFnRiSYFrAgMnRiSaCaChTiSnRiSaCaCrAFBiTiTrAFnRiSnRrAFnRiSaChTiSaCrAgMiTnRiSaCrAgMPBPBiTnRiShTiSaCBPrAFBiTiTPBiTrAFnRiSBnRiSaCnRC

In [ ]:
public Dictionary<string, List<int>> GetMatches(string input)
{
    var matches = new Dictionary<string, List<int>>();

    foreach(var key in d.Keys)
    {        
        var index = -1;
        while(true)
        {
            index = input.IndexOf(key, index + 1);
            
            if(index == -1)
            {
                break;
            }

            if(!matches.ContainsKey(key))
            {
                matches.Add(key, new List<int>());
            }

            matches[key].Add(index);
        }        
    }
    return matches;
}

In [ ]:
public Dictionary<string, List<int>> ReverseGetMatches(string input)
{
    var matches = new Dictionary<string, List<int>>();

    foreach(var key in rev.Keys)
    {        
        var index = -1;
        while(true)
        {
            index = input.IndexOf(key, index + 1);
            
            if(index == -1)
            {
                break;
            }

            if(!matches.ContainsKey(key))
            {
                matches.Add(key, new List<int>());
            }

            matches[key].Add(index);
        }
    }
    return matches;
}

In [ ]:
var matches = GetMatches(initial);

matches.Display();

key,value
Al,"[ 158, 225, 241 ]"
B,"[ 9, 19, 25, 30, 43, 45, 91, 143, 163, 196, 238, 245, 261, 267, 269, 294, 315, 339, 376, 417 ... (2 more) ]"
Ca,"[ 3, 31, 51, 63, 69, 95, 97, 107, 109, 127, 136, 151, 164, 166, 178, 187, 189, 207, 209, 211 ... (32 more) ]"
F,"[ 14, 26, 75, 84, 92, 119, 125, 129, 172, 182, 201, 276, 278, 291, 303, 305, 333, 345, 368, 370 ... (6 more) ]"
Mg,"[ 47, 59, 115, 147, 204, 327, 401, 440, 464 ]"
P,"[ 20, 29, 44, 46, 142, 144, 153, 162, 195, 237, 260, 266, 268, 314, 326, 336, 342, 400, 416, 432 ... (3 more) ]"
Si,"[ 5, 10, 33, 37, 53, 65, 71, 80, 99, 103, 111, 121, 132, 138, 156, 168, 174, 191, 197, 215 ... (21 more) ]"
Th,"[ 35, 67, 105, 134, 140, 176, 217, 256, 264, 283, 310, 318, 381, 386, 390, 412, 457 ]"
Ti,"[ 17, 21, 23, 41, 57, 87, 89, 145, 337, 396 ]"


In [ ]:
ReverseGetMatches(initial).Display();

key,value
ThF,[ 381 ]
BCa,"[ 30, 163, 245, 269, 294, 339, 376, 417 ]"
TiB,"[ 17, 23, 41, 89, 337 ]"
CaCa,"[ 95, 107, 164, 187, 207, 209, 211, 219, 229, 246, 248, 250, 295, 348, 350, 352, 360, 428, 435 ]"
PB,"[ 29, 44, 142, 162, 195, 237, 260, 266, 268, 314, 416, 461 ]"
PRnFAr,[ 342 ]
SiRnFYFAr,"[ 272, 299, 364, 444 ]"
SiRnMgAr,[ 111 ]
SiTh,"[ 33, 65, 103, 132, 138, 174, 215, 254, 262, 281, 308, 316, 379, 388, 410, 455 ]"
CaF,[ 127 ]


In [ ]:
matches.Values.Select(c => c.Count()).Sum().Display();


203

In [ ]:
var r = new Dictionary<int, int>();
foreach(var match in matches)
{
    var length = match.Key.Length;
    var total = initial.Length;
    foreach(var k in d[match.Key])
    {
        var newLength = k.Length;
        foreach(var v in match.Value)
        {
            r[(initial.Substring(0,v) + k + initial.Substring(v + length)).GetHashCode()] = 1;
        }
    }
}

In [ ]:
r.Count

509

In [ ]:
// var r = new Dictionary<int, int>();

// var target = "CRnCaSiRnBSiRnFArTiBPTiTiBFArPBCaSiThSiRnTiBPBPMgArCaSiRnTiMgArCaSiThCaSiRnFArRnSiRnFArTiTiBFArCaCaSiRnSiThCaCaSiRnMgArFYSiRnFYCaFArSiThCaSiThPBPTiMgArCaPRnSiAlArPBCaCaSiRnFYSiThCaRnFArArCaCaSiRnPBSiRnFArMgYCaCaCaCaSiThCaCaSiAlArCaCaSiRnPBSiAlArBCaCaCaCaSiThCaPBSiThPBPBCaSiRnFYFArSiThCaSiRnFArBCaCaSiRnFYFArSiThCaPBSiThCaSiRnPMgArRnFArPTiBCaPRnFArCaCaCaCaSiRnCaCaSiRnFYFArFArBCaSiThFArThSiThSiRnTiRnPMgArFArCaSiThCaPBCaSiRnBFArCaCaPRnCaCaPMgArSiRnFYFArCaSiThRnPBPMgAr".GetHashCode();
// var queue = new PriorityQueue<string, int>();
// queue.Enqueue("e",0);
// string s;

// var c = 0;

// var b = new StringBuilder();

// while(queue.TryDequeue(out s, out int i))
// {
//     c++;
//     var matches = GetMatches(s);
//     foreach(var match in matches)
//     {
//         var length = match.Key.Length;
//         var total = s.Length;
//         foreach(var k in d[match.Key])
//         {
//             var newLength = k.Length;
//             foreach(var v in match.Value)
//             {
//                 b.Clear();
//                 b.Append(s,0,v);
//                 b.Append(k);
//                 b.Append(s.Substring(v + length));
//                 var next = b.ToString();
//                 var hash = next.GetHashCode();
//                 if(hash == target)
//                 {
//                     Console.WriteLine($"Found match at {i}");
//                 }
//                 if(!r.ContainsKey(hash))
//                 {
//                     queue.Enqueue(next, i + 1);
//                     r[hash] = 1;
//                 }
//             }
//         }
//     }

//     if(c % 1000000 == 0)
//     {
//         s.Display();
//         break;
//     }
// }

In [ ]:
public class HoState : State
{    
    public bool IsGoal => Id == "e";
    public int Operations {get; init;}

    public string Id {get; init;}

    static StringBuilder b = new StringBuilder();

    public IEnumerable<(State state, long cost)> Neighbours()
    {
        var matches = ReverseGetMatches(Id);
        var total = Id.Length;
        foreach(var match in matches)
        {
            var length = match.Key.Length;            
            var k = rev[match.Key];
            
            var newLength = k.Length;
            foreach(var v in match.Value)
            {
                b.Clear();
                b.Append(Id,0,v);
                b.Append(k);
                b.Append(Id.Substring(v + length));
                var next = b.ToString();
                
                yield return (new HoState{Id = next, Operations = Operations + 1}, k.Length -length);
            }
        }
    }

    public Dictionary<string, List<int>> ReverseGetMatches(string input)
    {
        var matches = new Dictionary<string, List<int>>();

        foreach(var key in rev.Keys)
        {        
            var index = -1;
            while(true)
            {
                index = input.IndexOf(key, index + 1);
                
                if(index == -1)
                {
                    break;
                }

                if(!matches.ContainsKey(key))
                {
                    matches.Add(key, new List<int>());
                }

                matches[key].Add(index);
            }
        }
        return matches;
    }
}

In [ ]:
var aser = new Dijkstra();
var start = new HoState{Id = initial};

using(new Debug())
{
    aser.Run(start).Display();
}

aser.Count.Display();

Visited 100000 nodes. 10195 left, Current: CRnCaSiRnFYCaRnFArArPBPBPBPMgArThCaSiRnBFArCaRnPMgAr, Min: 23
Visited 200000 nodes. 10385 left, Current: CRnSiRnFYCaRnFArArCaPBPBPBPMgArThSiRnMgArSiRnMgArCaRnFAr, Min: 23
Visited 300000 nodes. 10274 left, Current: CRnCaCaSiRnFYCaRnFArArPBCaPBPBCaFArThCaSiRnMgArCaRnFAr, Min: 23
Visited 400000 nodes. 10216 left, Current: CRnSiRnMgArSiRnMgArSiRnFYCaRnFArArCaCaCaPBPMgArThSiRnBFArPBCaRnPMgAr, Min: 23
Visited 500000 nodes. 11303 left, Current: CRnSiRnBFArCaSiRnFYCaRnFArArCaPBPBCaPMgArThSiRnMgArPBSiRnBFArCaRnPBFAr, Min: 23
Visited 600000 nodes. 10714 left, Current: CRnSiRnBFArSiRnFYCaRnFArArPBPTiTiBPTiMgArThPBRnPBFAr, Min: 23
Visited 700000 nodes. 10517 left, Current: CRnSiRnMgArSiRnBFArSiRnFYCaRnFArArCaPBCaPTiTiBFArThCaRnPMgAr, Min: 23
Visited 800000 nodes. 10542 left, Current: CRnSiRnBFArSiRnMgArSiRnFYCaRnFArArPBPBPBPBCaCaFArThCaPBRnPMgAr, Min: 23
Visited 900000 nodes. 11152 left, Current: CRnCaSiRnMgArSiRnFYCaRnFArArPBPBPBPBPTiMgArThCaCaSiRnMgArCa

In [ ]:
aser.MinState

IsGoal,Operations,Id
False,190,CRnSiRnFYCaRnFArArFArAl


In [ ]:
aser.MinState.Neighbours()
// aser.Cost.Display();
// aser.Queue.Display();

(empty)

In [ ]:
// var r = new Dictionary<string, int>();

// var target = "CRnCaSiRnBSiRnFArTiBPTiTiBFArPBCaSiThSiRnTiBPBPMgArCaSiRnTiMgArCaSiThCaSiRnFArRnSiRnFArTiTiBFArCaCaSiRnSiThCaCaSiRnMgArFYSiRnFYCaFArSiThCaSiThPBPTiMgArCaPRnSiAlArPBCaCaSiRnFYSiThCaRnFArArCaCaSiRnPBSiRnFArMgYCaCaCaCaSiThCaCaSiAlArCaCaSiRnPBSiAlArBCaCaCaCaSiThCaPBSiThPBPBCaSiRnFYFArSiThCaSiRnFArBCaCaSiRnFYFArSiThCaPBSiThCaSiRnPMgArRnFArPTiBCaPRnFArCaCaCaCaSiRnCaCaSiRnFYFArFArBCaSiThFArThSiThSiRnTiRnPMgArFArCaSiThCaPBCaSiRnBFArCaCaPRnCaCaPMgArSiRnFYFArCaSiThRnPBPMgAr".GetHashCode();
// var queue = new PriorityQueue<string, int>();
// queue.Enqueue("e",0);
// string s;

// var c = 0;

// var b = new StringBuilder();

// while(queue.TryDequeue(out s, out int i))
// {
//     c++;
//     var matches = GetMatches(s);
//     foreach(var match in matches)
//     {
//         var length = match.Key.Length;
//         var total = s.Length;
//         foreach(var k in d[match.Key])
//         {
//             var newLength = k.Length;
//             foreach(var v in match.Value)
//             {
//                 b.Clear();
//                 b.Append(s,0,v);
//                 b.Append(k);
//                 b.Append(s.Substring(v + length));
//                 var next = b.ToString();
//                 var hash = next.GetHashCode();
//                 if(hash == target)
//                 {
//                     Console.WriteLine($"Found match at {i}");
//                 }
//                 if(!r.ContainsKey(hash))
//                 {
//                     queue.Enqueue(next, i + 1);
//                     r[hash] = 1;
//                 }
//             }
//         }
//     }

//     if(c % 1000000 == 0)
//     {
//         s.Display();
//         break;
//     }
// }